<h1> Salesforce/blip-image-captioning-large </h1>

In [1]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device.type

'cuda'

In [4]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

2024-07-17 17:08:43.940312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 17:08:43.940421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 17:08:44.091465: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [5]:
# Saving the model

save_directory = "blip-image-captioning-large"
model.save_pretrained(save_directory)

In [6]:
# Loading the model

loaded_model = BlipForConditionalGeneration.from_pretrained(save_directory)

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
loaded_model = loaded_model.to(device.type)

In [8]:
img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

In [9]:
# conditional image captioning
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt").to(device.type)

out = loaded_model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a photography of a woman and her dog on the beach


In [10]:
# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt").to(device.type)

out = loaded_model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

woman sitting on the beach with her dog and a cell phone


In [11]:
def generate_caption(url):
    raw_image = Image.open(requests.get(url, stream=True).raw).convert('RGB')
    inputs = processor(raw_image, return_tensors="pt").to(device.type)

    out = loaded_model.generate(**inputs)
    print(processor.decode(out[0], skip_special_tokens=True))

In [12]:
generate_caption('https://images8.alphacoders.com/130/thumb-1920-1300450.jpg')

spider - man swimming in the ocean with a fish


In [13]:
generate_caption('https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTYDKOY_SMMxCWDsG-gl0Yzt9-4LeVp5iLfGQ&s')

dragon ball gohan eating a sandwich with a plate of fries
